# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [2]:
# Your code goes here
import pandas as pd
import numpy as np

files = [
    "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv",
    "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv",
    "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv",
]
data_list = [pd.read_csv(f) for f in files]
for csv in data_list:
    csv.dropna(axis=0, how="all", inplace=True)


In [3]:
for df in data_list:
    df.columns = [item.lower().replace(' ', '_') for item in df.columns]
    df.columns = [item.replace('st', 'state') if item == 'st' else item for item in df.columns]

In [4]:
data = pd.concat(data_list, ignore_index=True)

In [5]:
len(data) - len(data.drop_duplicates())

3

In [6]:
#DEFINING FUNCTIONS

def convert_lt_value(element):
    if element == np.nan:
        return np.nan
    if type(element)==float:
        return element
    if element[-1] == "%":
        return float(element[:-1])/100
    
def convert_num_complaints(element):
    if type(element) != str:
        return np.nan
    if element[2].isdigit():
        return int(element[2])
    else:
        return np.nan

def convert_gender(element):
    if type(element) == float:
        return np.nan
    if element[0].lower() == "f":
        return "f"
    if element[0].lower() == "m":
        return "m"
    else:
        return np.nan

def convert_income(element):
    if element == 0:
        return np.nan
    else:
        return element
    
def convert_st(element):
    return element[0:2].upper()

def convert_edu(element):
    if element == "Bachelors":
        return "Bachelor"
    return element.strip()

def convert_vehicleclass(element):
    if element in["Sports Car", "Luxury SUV", "Luxury Car" ]:
        return "Luxury"
    return element


In [8]:
#CALLING FUNCTIONS

#NUMERICAL
data["customer_lifetime_value_r"] = data["customer_lifetime_value"].apply(convert_lt_value)
data["number_of_open_complaints_r"] = data["number_of_open_complaints"].apply(convert_num_complaints)
data["income_r"] = data["income"].apply(convert_income)


#CATEGORICAL
data["gender_r"] = data["gender"].apply(convert_gender)
data["state_r"] = data["state"].apply(convert_st)
data["education_r"] = data["education"].apply(convert_edu)
data["vehicle_class_r"] =data["vehicle_class"].apply(convert_vehicleclass)

In [9]:
print(data.columns)
data.shape

Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'vehicle_class', 'total_claim_amount',
       'customer_lifetime_value_r', 'number_of_open_complaints_r', 'income_r',
       'gender_r', 'state_r', 'education_r', 'vehicle_class_r'],
      dtype='object')


(9137, 18)

In [16]:
len(data["customer"].unique())

9056

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [10]:
marketing_data = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv")
marketing_data.dropna(axis=0, how="all", inplace=True)

In [11]:
marketing_data.columns

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')

In [17]:
len(marketing_data["customer"].unique())

9134

In [51]:
#INNER JOIN
print("Initial dataframe size: ", data.shape, "unique customer ID: ", len(data["customer"].unique()))
print("Duplicates in data: ",len(data) - len(data.drop_duplicates(subset=["customer"])))
print("Marketing dataframe size ", marketing_data.shape, "unique customer ID: ", len(marketing_data["customer"].unique()))
print("Duplicates in marketing data: ", len(marketing_data) - len(marketing_data.drop_duplicates(subset=["customer"])))
merged_df = pd.merge(data, marketing_data, on='customer', how='left')
print("merged result ", merged_df.shape, "unique customer ID: ", len(data["customer"].unique()))
print("Duplicates in marketing data: ", len(merged_df) - len(merged_df.drop_duplicates(subset=["customer"])))

Initial dataframe size:  (9137, 18) unique customer ID:  9056
Duplicates in data:  81
Marketing dataframe size  (10910, 27) unique customer ID:  9134
Duplicates in marketing data:  1776
merged result  (10910, 44) unique customer ID:  9056
Duplicates in marketing data:  1854


1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [59]:
marketing_data.pivot_table(values="income", index="sales_channel", aggfunc="mean").round(2)

,income
sales_channel,
Agent,37003.19
Branch,37649.11
Call Center,37858.48
Web,38253.45


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [58]:
marketing_data.pivot_table(values="customer_lifetime_value", index=["gender", "education"], aggfunc="mean").round(2)

customer_lifetime_value
gender education                                    
F      Bachelor                              7874.27
       College                               7748.82
       Doctor                                7328.51
       High School or Below                  8675.22
       Master                                8157.05
M      Bachelor                              7703.60
       College                               8052.46
       Doctor                                7415.33
       High School or Below                  8149.69
       Master                                8168.83

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here